In [89]:
!git clone https://github.com/khhha-pffuuuu/ITMO_Competition.git


Cloning into 'ITMO_Competition'...
remote: Enumerating objects: 49, done.
remote: Total 49 (delta 0), reused 0 (delta 0), pack-reused 49 (from 1)
Receiving objects: 100% (49/49), 42.10 MiB | 18.79 MiB/s, done.
Resolving deltas: 100% (9/9), done.


In [90]:
cd ITMO_Competition/

/content/ITMO_Competition/ITMO_Competition/ITMO_Competition


In [91]:
!git checkout ML-branch

Branch 'ML-branch' set up to track remote branch 'ML-branch' from 'origin'.
Switched to a new branch 'ML-branch'


In [92]:
!pip install -r requirements.txt

In [93]:
import pandas as pd

data = pd.read_excel('/content/ITMO_Competition/data/aug_data.xlsx')
test_data = pd.read_excel('/content/ITMO_Competition/data/dataset_comments_79.xlsx')

In [94]:
test_data.head()

,UserSenderId,SubmitDate,MessageText,labels
0,16899,2021-03-15 19:18:35.977,"<p class=""st0"">Алексей, С Днем Рождения!!! Исп...",G
1,17979,2021-03-15 17:47:22.810,"<p class=""st0"">С Днём Рождения!</p>",G
2,8492,2021-03-15 17:12:26.303,"<p class=""st0"">Староста, с днюхой!<br></p>",G
3,8562,2021-03-15 15:06:22.740,"<p class=""st0"">Коля, мои поздравления! Желаю х...",G
4,8562,2021-03-15 14:58:11.817,"<p class=""st0"">Леша, поздравляю! Всех благ! </p>",G


In [95]:
data.head()

,MessageText,labels
0,В общем podman - это не наш метод. На нем рабо...,0
1,"А зачем ввод ИНН? Если его нет у компании, то ...",1
2,сориентируйте по размерам пожалуйста,1
3,"толстовка хорошая, но начес лезет сильно. над...",2
4,Здорово! Нащупали идеальный формат 😎👍👍 Из пред...,2


In [96]:
from bs4 import BeautifulSoup
import re

# Скопируй функции из data_preprocessing.ipynb
def html_to_text(html):
    soup = BeautifulSoup(html, "html.parser")
    text = soup.get_text(separator=" ")
    text = text.replace("\xa0", " ")
    return text.strip()

def is_noise(text):
    if len(text) < 3:
        return True
    if re.match(r"^[\W\d_]+$", text):
        return True
    if re.match(r"^[a-zA-Z]+$", text):
        return True
    return False

In [97]:
data.MessageText = data.MessageText.apply(lambda x: html_to_text(x))
data = data[~data.MessageText.apply(is_noise)]

In [98]:
test_data.drop(['UserSenderId', 'SubmitDate'], axis=1)

,MessageText,labels
0,"<p class=""st0"">Алексей, С Днем Рождения!!! Исп...",G
1,"<p class=""st0"">С Днём Рождения!</p>",G
2,"<p class=""st0"">Староста, с днюхой!<br></p>",G
3,"<p class=""st0"">Коля, мои поздравления! Желаю х...",G
4,"<p class=""st0"">Леша, поздравляю! Всех благ! </p>",G
...,...,...
73,"<p class=""st0"">Кажется, все идет не по плану.</p>",B
74,"<p class=""st0"">Разочарован, ожидал лучшего.</p>",B
75,"<p class=""st0"">Почему всегда так сложно?</p>",B
76,"<p class=""st0"">Ничего не меняется, увы.</p>",B


In [99]:
test_data.MessageText = test_data.MessageText.apply(lambda x: html_to_text(x))
test_data = test_data[~test_data.MessageText.apply(is_noise)]
test_data.drop(['UserSenderId', 'SubmitDate'], axis=1)

,MessageText,labels
0,"Алексей, С Днем Рождения!!! Исполнения всего з...",G
1,С Днём Рождения!,G
2,"Староста, с днюхой!",G
3,"Коля, мои поздравления! Желаю хороших отпусков...",G
4,"Леша, поздравляю! Всех благ!",G
...,...,...
73,"Кажется, все идет не по плану.",B
74,"Разочарован, ожидал лучшего.",B
75,Почему всегда так сложно?,B
76,"Ничего не меняется, увы.",B


In [100]:
dict_classes = {
    'G': 2,
    'N': 1,
    'B': 0
}

test_data.labels = test_data.labels.apply(lambda x: dict_classes[x])

In [101]:
def preprocess_function(examples):
    inputs = tokenizer(
        examples['MessageText'],
        truncation=True,
        padding='max_length',
        max_length=128,
        return_tensors="pt"
    )

    return inputs

train_dataset = Dataset.from_pandas(data).map(preprocess_function, batched=True)
test_dataset = Dataset.from_pandas(test_data).map(preprocess_function, batched=True)

Map:   0%|          | 0/1913 [00:00<?, ? examples/s]

Map:   0%|          | 0/78 [00:00<?, ? examples/s]

In [102]:
test_data.head()

,UserSenderId,SubmitDate,MessageText,labels
0,16899,2021-03-15 19:18:35.977,"Алексей, С Днем Рождения!!! Исполнения всего з...",2
1,17979,2021-03-15 17:47:22.810,С Днём Рождения!,2
2,8492,2021-03-15 17:12:26.303,"Староста, с днюхой!",2
3,8562,2021-03-15 15:06:22.740,"Коля, мои поздравления! Желаю хороших отпусков...",2
4,8562,2021-03-15 14:58:11.817,"Леша, поздравляю! Всех благ!",2


In [103]:
!pip install datasets

In [104]:
from datasets import Dataset

def preprocess_function(examples):
    inputs = tokenizer(
        examples['MessageText'],
        truncation=True,
        padding='max_length',
        max_length=128,
        return_tensors="pt"
    )

    return inputs

train_dataset = Dataset.from_pandas(data).map(preprocess_function, batched=True)
test_dataset = Dataset.from_pandas(test_data).map(preprocess_function, batched=True)

Map:   0%|          | 0/1913 [00:00<?, ? examples/s]

Map:   0%|          | 0/78 [00:00<?, ? examples/s]

In [105]:
from dataclasses import dataclass

@dataclass
class Config:
    model_name = "cointegrated/rubert-tiny-sentiment-balanced"  # "blanchefort/rubert-base-cased-sentiment"
    new_model = "seq-cls-ft-system"
    wb_token = '97ccf181ecf3a5c0953f355390d1e30fc03c3f0a'

config = Config()

In [106]:
import wandb

# Зайдем в W&B
wandb.login(key=config.wb_token)

run = wandb.init(
    project='Fine-tune Pre-Trained SEQ-CLS',
    job_type="training"
)


wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


In [107]:
from torch import nn

# import bitsandbytes as bnb
from transformers import (Trainer, TrainingArguments,
                          AutoModelForSequenceClassification,
                          AutoTokenizer, EarlyStoppingCallback)

# Модель
tokenizer = AutoTokenizer.from_pretrained(config.model_name)
model = AutoModelForSequenceClassification.from_pretrained(config.model_name)

# Заменяем последний слой и остальные замораживаем
for name, param in model.named_parameters():
    if "bert.encoder.layer.1" in name:  # "bert.encoder.layer.9" in name
        break
    else:
        param.requires_grad = False

In [108]:
model_path = 'https://github.com/khhha-pffuuuu/ITMO_Competition/tree/ML-branch/models/model'
tokenizer_path = 'https://github.com/khhha-pffuuuu/ITMO_Competition/tree/ML-branch/models/tokenizer'

In [109]:
tokenizer = AutoTokenizer.from_pretrained(config.model_name)
model = AutoModelForSequenceClassification.from_pretrained(config.model_name)

In [110]:
import torch
import numpy as np
from sklearn.metrics import recall_score

# Переводим модель в режим оценки
model.eval()

# Подготовка данных
input_ids = torch.tensor(test_dataset['input_ids']).to(model.device)
attention_mask = torch.tensor(test_dataset['attention_mask']).to(model.device)
true_labels = torch.tensor(test_dataset['labels']).cpu().numpy()

# Получаем предсказания
with torch.no_grad():  # Отключаем вычисление градиентов
    outputs = model(input_ids=input_ids, attention_mask=attention_mask)
    logits = outputs.logits

# Преобразуем логиты в предсказанные классы
predicted_labels = torch.argmax(logits, dim=-1).cpu().numpy()

# Вычисляем recall
recall = recall_score(true_labels, predicted_labels, average=None)

# Вывод recall для каждого класса
print("Recall для каждого класса:")
for i, rec in enumerate(recall):
    print(f"Класс {i}: {rec:.2f}")

# Вывод macro-averaged recall
macro_recall = recall_score(true_labels, predicted_labels, average="macro")
print(f"\nMacro-averaged recall: {macro_recall:.2f}")

# Вывод weighted-averaged recall
weighted_recall = recall_score(true_labels, predicted_labels, average="weighted")
print(f"Weighted-averaged recall: {weighted_recall:.2f}")

Recall для каждого класса:
Класс 0: 0.50
Класс 1: 0.88
Класс 2: 0.84

Macro-averaged recall: 0.74
Weighted-averaged recall: 0.76


In [111]:
import torch
import asyncio
from IPython.display import display, clear_output
import ipywidgets as widgets
from torch import nn
from sklearn.metrics import recall_score
import random

# Определяем устройство
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Определяем функцию потерь и оптимизатор
criterion = nn.CrossEntropyLoss()  # Функция потерь для многоклассовой классификации
optimizer = torch.optim.Adam(model.parameters(), lr=7e-6)  # Оптимизатор

# Создаем интерактивные элементы
output = widgets.Output()
feedback_event = asyncio.Event()  # Событие для ожидания фидбэка
user_feedback = -1


In [112]:
# Функция для ввода пользователя
def get_user_feedback():
    while True:
        try:
            feedback = int(input("Введите класс (0, 1, 2): "))
            if feedback in [0, 1, 2]:
                return feedback
            else:
                print("Некорректный ввод. Введите 0, 1 или 2.")
        except ValueError:
            print("Некорректный ввод. Введите число.")

In [23]:
import random
from sklearn.metrics import recall_score
import torch
from IPython.display import display, clear_output
import builtins

# Включаем градиенты для классификатора
for param in model.classifier.parameters():
    param.requires_grad = True

# Модифицированная функция обучения
async def async_rlhf_training(model, train_dataset, test_dataset, num_epochs=1, feedback_interval=4):
    model.train()
    display(output)

    # Инициализируем оптимизатор
    optimizer = torch.optim.Adam(
        filter(lambda p: p.requires_grad, model.parameters()),
        lr=1e-5
    )

    # Выбираем 10 случайных записей
    random_indices = random.sample(range(len(train_dataset)), 10)
    random_samples = [train_dataset[i] for i in random_indices]

    # Для сбора итоговых результатов
    final_recall = None

    for i, batch in enumerate(random_samples):
        global user_feedback
        user_feedback = -1
        feedback_event.clear()

        # Получаем предсказание
        text = batch["MessageText"]
        inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
        inputs = {k: v.to(device) for k, v in inputs.items()}

        with torch.set_grad_enabled(True):
            outputs = model(**inputs)

        prediction = torch.argmax(outputs.logits, dim=-1).item()

        with output:
            clear_output(wait=True)
            print(f"Текст: {text}")
            print(f"Предсказание модели: {prediction}")

        # Получаем фидбэк
        def get_user_feedback():
            feedback = int(builtins.input("Введите класс (0, 1, 2): "))

            #feedback = int(input("Введите класс (0, 1, 2): "))
            if feedback in [0, 1, 2]:
                return feedback
            else:
                print("Некорректный ввод. Пожалуйста, введите 0, 1 или 2.")
                return get_user_feedback()

        user_feedback = get_user_feedback()
        feedback_event.set()

        # Обучение
        target = torch.tensor([user_feedback], device=device)
        loss = criterion(outputs.logits, target)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Оценка каждые feedback_interval примеров
        if (i + 1) % feedback_interval == 0:
            model.eval()
            all_labels = []
            all_preds = []

            for test_batch in test_dataset:
                inputs = tokenizer(test_batch["MessageText"],
                                 return_tensors="pt",
                                 padding=True,
                                 truncation=True)
                inputs = {k: v.to(device) for k, v in inputs.items()}

                with torch.no_grad():
                    outputs = model(**inputs)

                preds = torch.argmax(outputs.logits, dim=-1).cpu().numpy()

                # Обработка меток
                if isinstance(test_batch["labels"], (int, float)):
                    all_labels.append(test_batch["labels"])
                else:
                    all_labels.extend(test_batch["labels"].tolist())

                all_preds.extend(preds.tolist())

            recall = recall_score(all_labels, all_preds, average="macro")
            final_recall = recall  # Сохраняем последний результат

            with output:
                clear_output(wait=True)
                print(f"Recall на тестовом наборе: {recall:.4f}")

            model.train()

    # Возвращаем итоговый recall
    return final_recall

In [22]:
# Запуск обучения
import nest_asyncio
nest_asyncio.apply()

recall = await async_rlhf_training(
    model=model,
    train_dataset=train_dataset,
    test_dataset=test_dataset,
    num_epochs=3,
    feedback_interval=4
)
print(f"Финальный recall: {recall:.4f}")

In [113]:
# import random
# from sklearn.metrics import recall_score
# import torch
# from IPython.display import display, clear_output

# for param in model.classifier.parameters():
#     param.requires_grad = True

# # Модифицированная функция обучения
# async def async_rlhf_training(model, train_dataset, test_dataset, num_epochs=1, feedback_interval=4):
#     model.train()
#     display(output)

#     # Инициализируем оптимизатор
#     optimizer = torch.optim.Adam(
#         filter(lambda p: p.requires_grad, model.parameters()),
#         lr=1e-5
#     )

#     # Выбираем 10 случайных записей
#     random_indices = random.sample(range(len(train_dataset)), 10)
#     random_samples = [train_dataset[i] for i in random_indices]

#     # Для сбора итоговых результатов
#     final_recall = None

#     for i, batch in enumerate(random_samples):
#         global user_feedback
#         user_feedback = -1
#         feedback_event.clear()

#         # Получаем предсказание
#         text = batch["MessageText"]
#         inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
#         inputs = {k: v.to(device) for k, v in inputs.items()}

#         with torch.set_grad_enabled(True):
#             outputs = model(**inputs)

#         prediction = torch.argmax(outputs.logits, dim=-1).item()

#         with output:
#             clear_output(wait=True)
#             print(f"Текст: {text}")
#             print(f"Предсказание модели: {prediction}")

#         # Получаем фидбэк
#         user_feedback = get_user_feedback()
#         feedback_event.set()

#         # Обучение
#         target = torch.tensor([user_feedback], device=device)
#         loss = criterion(outputs.logits, target)

#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()

#         # Оценка каждые feedback_interval примеров
#         if (i + 1) % feedback_interval == 0:
#             model.eval()
#             all_labels = []
#             all_preds = []

#             for test_batch in test_dataset:
#                 inputs = tokenizer(test_batch["MessageText"],
#                                  return_tensors="pt",
#                                  padding=True,
#                                  truncation=True)
#                 inputs = {k: v.to(device) for k, v in inputs.items()}

#                 with torch.no_grad():
#                     outputs = model(**inputs)

#                 preds = torch.argmax(outputs.logits, dim=-1).cpu().numpy()

#                 # Обработка меток
#                 if isinstance(test_batch["labels"], (int, float)):
#                     all_labels.append(test_batch["labels"])
#                 else:
#                     all_labels.extend(test_batch["labels"].tolist())

#                 all_preds.extend(preds.tolist())

#             recall = recall_score(all_labels, all_preds, average="macro")
#             final_recall = recall  # Сохраняем последний результат

#             with output:
#                 clear_output(wait=True)
#                 print(f"Recall на тестовом наборе: {recall:.4f}")

#             model.train()

#     # Возвращаем итоговый recall
#     return final_recall

In [21]:
# # Запуск обучения
# import nest_asyncio
# recall = await async_rlhf_training(
#     model=model,
#     train_dataset=train_dataset,
#     test_dataset=test_dataset,
#     num_epochs=3,
#     feedback_interval=4
# )
# print(f"Финальный recall: {recall:.4f}")